In [70]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from deep_translator import GoogleTranslator

Retrieving data

In [245]:
hourly_url = "https://api.ouka.fi/v1/properties_consumption_hourly"
def retrievePropertiesConsumptionByYear(ids, years):
    """
    Return a dataframe containing the energy consumption of properties during a some years
    """
    def my_query(args):
        """
        Format a query using args: (id, year).
        """
        query = (hourly_url
                + "?property_id=eq."
                + str(args[0])
                + "&year=eq."
                + str(args[1])
                )
        return query

    q_args = product(ids, years)
    # form a query per each id+year combination:
    queries = [my_query(p) for p in q_args]
    dfs = [pd.read_json(q) for q in queries] # fetch data for each query

    return pd.concat(dfs)

In [ ]:
ids = [657701]
years = [2019]
dfs_all = retrievePropertiesConsumptionByYear(ids, years)

Time type preprocessing

In [248]:
from datetime import datetime

def replaceTime(df):
    """
    replace the four columns of time (year, month, day and hour) by only one normalized column using datetime library
    """
    
    # Saving times
    timeDf = df[["year", "month", "day", "starting_hour"]].values.tolist()
    
    # COnverting to datetime type
    new_time_list = [datetime(time[0], time[1], time[2], time[3]) for time in timeDf]
    
    # Deleting obsolete columns
    df.drop("year", axis=1, inplace=True)
    df.drop("month", axis=1, inplace=True)
    df.drop("day", axis=1, inplace=True)
    df.drop("starting_hour", axis=1, inplace=True)
    
    # Putting the new normalized column to the df
    df["datetime"]=new_time_list
    df.sort_values(by="datetime")
    

Selecting a observation time

In [285]:
def selectTimeObservation(df, start_date, end_date):
    return df.loc[(df['datetime']>=start_time) & (df['datetime']<end_time)]

Overall retrieval function

In [287]:
from dateutil.relativedelta import relativedelta

def retrieveData(property_ids, start_date, end_date, energy_type):
    
    start_year = start_date.year
    end_year = end_date.year
    year_difference = end_year-start_year
    
    years = []
    for y in range(year_difference+1):
        years.append(start_year+y)
    
    
    bas_df = retrievePropertiesConsumptionByYear(property_ids, years)
    
    replaceTime(bas_df)
    
    return selectTimeObservation(bas_df, start_date, end_date)

In [292]:
start_date = datetime(2019, 1, 1, 0)
end_date = datetime(2019, 1, 2, 0)
ids = [657701]

final_df = retrieveData(ids, start_date, end_date, "c")

In [293]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 632 to 2115
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   property_id           48 non-null     int64         
 1   property_internal_id  48 non-null     object        
 2   property_name         48 non-null     object        
 3   consumption_measure   48 non-null     object        
 4   consumption           48 non-null     float64       
 5   keyfield              48 non-null     int64         
 6   datetime              48 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 4.0+ KB


Translation